# 安装环境和包

In [ ]:
!pip install pandas openpyxl akshare

  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 28.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.1/10.1 MB 49.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.4/5.4 MB 94.3 MB/s eta 0:00:00
  Created wheel for jsonpath: filename=jsonpath-0.82.2-py3-none-any.whl size=5615 sha256=956cd80991d3131cb26e4c359a2006825cb16c5c2ea8eed05d1bde37b51b29dc
  Stored in directory: /root/.cache/pip/wheels/7d/b8/16/48bbd715040679311fa68cb564ad24a97d9a67fea5d4c848c8
Successfully built jsonpath


In [ ]:
import akshare as ak
import pandas as pd
import os
from tqdm import tqdm  # 进度条支持

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# 找出每个公司过去的年报pdf

In [ ]:
pre_dir="./"
symbol_str = '''688981,603993,603259,601899,601888,601816,601766,601668,601600,601318,601088,601012,600941,600900,600893,600887,600660,600585,600519,600436,600426,600415,600406,600309,600276,600176,600036,600031,600030,600028,600019,600009,300760,300750,300408,300124,300122,300015,002714,002594,002475,002371,002230,002027,000938,000792,000725,000333,000063,000002'''

In [ ]:
HuShen300 = '''688981,688599,688506,688472,688396, 688303, 688271, 688256, 688223, 688187,688169, 688126, 688111, 688082, 688041, 688036, 688012, 688009, 688008, 605499,605117, 603993, 603986, 603833, 603806, 603799, 603659, 603501, 603392, 603369,603296, 603288, 603260, 603259, 603195, 603019, 601998, 601995, 601989, 601988,601985, 601939, 601919, 601916, 601901, 601899, 601898, 601888, 601881, 601878,601877, 601872, 601868, 601865, 601857, 601838, 601818, 601816, 601808, 601800,601799, 601788, 601766, 601728, 601699, 601698, 601689, 601688, 601669, 601668,601658, 601633, 601628, 601618, 601607, 601601, 601600, 601398, 601390, 601377,601360, 601336, 601328, 601319, 601318, 601288, 601238, 601236, 601229, 601225,601211, 601186, 601169, 601166, 601138, 601136, 601127, 601117, 601111, 601100,601088, 601066, 601059, 601058, 601021, 601012, 601009, 601006, 600999, 600989,600958, 600941, 600938, 600926, 600919, 600918, 600905, 600900, 600893, 600887,600886, 600875, 600845, 600809, 600803, 600795, 600760, 600745, 600741, 600690,600674, 600660, 600600, 600588, 600585, 600584, 600570, 600547, 600519, 600515,600489, 600482, 600460, 600438, 600436, 600426, 600415, 600406, 600377, 600372,600362, 600346, 600332, 600309, 600276, 600233, 600219, 600196, 600188, 600183,600176, 600161, 600160, 600150, 600115, 600111, 600104, 600089, 600085, 600066,600061, 600050, 600048, 600039, 600036, 600031, 600030, 600029, 600028, 600027,600026, 600025, 600023, 600019, 600018, 600016, 600015, 600011, 600010, 600009,600000, 301269, 300999, 300979, 300896, 300832, 300782, 300760, 300759, 300750,300661, 300628, 300502, 300498, 300450, 300442, 300433, 300418, 300413, 300408,300394, 300347, 300316, 300308, 300274, 300124, 300122, 300059, 300033, 300015,300014, 003816, 002938, 002920, 002916, 002812, 002736, 002714, 002709, 002648,002601, 002594, 002555, 002493, 002475, 002466, 002463, 002460, 002459, 002422,002415, 002371, 002352, 002311, 002304, 002271, 002252, 002241, 002236, 002230,002180, 002179, 002142, 002129, 002074, 002050, 002049, 002028, 002027, 002007,002001, 001979, 001965, 001289, 000999, 000983, 000977, 000975, 000963, 000938,000895, 000876, 000858, 000807, 000800, 000792, 000786, 000776, 000768, 000725,000708, 000661, 000651, 000630, 000625, 000617, 000596, 000568, 000538, 000425,000408, 000338, 000333, 000301, 000166, 000157, 000100, 000063, 000002, 000001'''

In [ ]:
def download_annual_reports(symbol_str: str, pre_dir: str):
    """
    批量下载股票年报数据

    参数：
    symbol_str : str - 逗号分隔的股票代码字符串
    pre_dir    : str - 文件保存目录，默认当前目录

    返回：
    error_list : list - 下载失败的股票代码列表
    """
    # 创建存储目录
    os.makedirs(pre_dir, exist_ok=True)

    # 拆分股票代码并去重
    symbols = list(set(symbol_str.replace(' ','').split(',')))
    # print(symbols)
    error_list = []

    # 带进度条的遍历
    for symbol in tqdm(symbols, desc="下载年报数据"):
        try:
            # print(symbol)
            df = ak.stock_zh_a_disclosure_report_cninfo(symbol=symbol, market="沪深京",
                                                        category="年报",
                                                        start_date="20200101",
                                                        end_date="20250515")

            df.to_excel(pre_dir+symbol+'.xlsx',engine='openpyxl')
        except:
            print(symbol)
    pass

In [ ]:
download_annual_reports(HuShen300, '/content/drive/MyDrive/513/HuShen300/')

下载年报数据: 100%|██████████| 300/300 [11:52<00:00,  2.37s/it]


In [ ]:
download_annual_reports(symbol_str, '/content/drive/MyDrive/513/A50/')

下载年报数据: 100%|██████████| 50/50 [02:03<00:00,  2.47s/it]


## 数据清洗
### 需要把年报公告中有‘摘要’，‘英文’的公告都去除掉

In [ ]:
def process_reports(folder_path: str, output_path: str):
    """
    批量处理文件夹中所有xlsx文件，保留纯年度报告数据
    :param folder_path: 包含xlsx文件的文件夹路径
    """
    # 获取文件列表（仅当前目录）
    files = [f for f in os.listdir(folder_path)
            if f.endswith('.xlsx') and not f.startswith('~$')]

    for file in files:
        try:
            # 构建完整路径
            file_path = os.path.join(folder_path, file)

            # 读取数据（网页6推荐的pandas方法）
            df = pd.read_excel(file_path, engine='openpyxl')

            # 核心筛选逻辑（网页7的布尔索引技巧）
            mask = (
                df['公告标题'].str.contains('年度报告', na=False) &
                ~df['公告标题'].str.contains('摘要|英文', regex=True, na=False)
            )
            filtered_df = df[mask]

            # 覆盖原文件保存（网页5的保存方式）
            filtered_df.to_excel(os.path.join(output_path, file), index=False, engine='openpyxl')

            print(f"成功处理：{file}")

        except Exception as e:
            print(f"处理失败：{file} | 错误：{str(e)}")


In [ ]:
input_folder = "/content/drive/MyDrive/513/HuShen300/"  # 原始文件目录
output_folder = "/content/drive/MyDrive/513/HuShen300_filtered/"

process_reports(input_folder, output_folder)

成功处理：601211.xlsx
成功处理：600176.xlsx
成功处理：605117.xlsx
成功处理：600674.xlsx
成功处理：600362.xlsx
成功处理：000895.xlsx
成功处理：601658.xlsx
成功处理：002415.xlsx
成功处理：601985.xlsx
成功处理：601088.xlsx
成功处理：601877.xlsx
成功处理：600886.xlsx
成功处理：601799.xlsx
成功处理：688981.xlsx
成功处理：600061.xlsx
成功处理：002601.xlsx
成功处理：601360.xlsx
成功处理：600958.xlsx
成功处理：002180.xlsx
成功处理：603296.xlsx
成功处理：600026.xlsx
成功处理：000408.xlsx
成功处理：600489.xlsx
成功处理：300394.xlsx
成功处理：688169.xlsx
成功处理：300661.xlsx
成功处理：300014.xlsx
成功处理：000708.xlsx
成功处理：002001.xlsx
成功处理：600000.xlsx


In [ ]:
input_folder = "/content/drive/MyDrive/513/A50/"  # 原始文件目录
output_folder = "/content/drive/MyDrive/513/A50_filtered/"

process_reports(input_folder, output_folder)

# 把查看pdf的url转换成下载的url

In [ ]:
def process_announcement_urls(source_dir: str, target_dir: str):
    """
    处理公告链接并生成PDF链接列表
    步骤：
    1. 遍历源目录中的Excel文件
    2. 从每个文件中提取公告链接
    3. 转换为PDF格式的URL
    4. 将结果保存到目标目录的文本文件中
    """
    # 配置路径（使用原始字符串防止转义）
    source_dir = source_dir
    target_dir = target_dir

    # 确保目标目录存在
    os.makedirs(target_dir, exist_ok=True)

    # 遍历源目录下所有文件
    for filename in os.listdir(source_dir):
        # 从文件名提取股票代码（前6位字符）
        ticker = filename[:6]

        try:
            # 构建完整文件路径并读取Excel
            file_path = os.path.join(source_dir, filename)
            df = pd.read_excel(file_path, engine='openpyxl')

            # 验证包含必要列
            if '公告链接' not in df.columns:
                print(f"文件 {filename} 缺少'公告链接'列，跳过处理")
                continue

            # 转换URL处理逻辑
            pdf_urls = []
            for url in df['公告链接'].tolist():
                # 参数安全解析
                params = url.split('&')
                if len(params) < 2:
                    print(f"非常规URL格式: {url}")
                    continue

                # 提取公告ID和时间参数
                try:
                    announcement_id = params[1].split('=')[1]  # 格式：announcementId=xxxxxx
                    raw_time = params[-1].split('=')[1]        # 格式：announcementTime=YYYY-MM-DD...
                    formatted_time = raw_time[:10]            # 取日期部分
                except IndexError as e:
                    print(f"参数解析失败: {url} ({e})")
                    continue

                # 生成标准PDF链接
                pdf_url = f'http://static.cninfo.com.cn/finalpage/{formatted_time}/{announcement_id}.PDF'
                pdf_urls.append(pdf_url)

            # 写入结果文件
            output_path = os.path.join(target_dir, f"{ticker}.txt")
            with open(output_path, 'w', encoding='utf-8') as f:
                f.write('\n'.join(pdf_urls))

            print(f"成功处理: {filename} => {ticker}.txt")

        except Exception as e:
            print(f"处理文件 {filename} 时发生错误: {str(e)}")
            continue


In [ ]:
source_dir="/content/drive/MyDrive/513/HuShen300_filtered/"
target_dir="/content/drive/MyDrive/513/HuShen300_pdf_download_links/"

process_announcement_urls(source_dir, target_dir)

NameError: name 'process_announcement_urls' is not defined

In [ ]:
source_dir="/content/drive/MyDrive/513/A50_filtered/"
target_dir="/content/drive/MyDrive/513/A50_pdf_download_links/"

process_announcement_urls(source_dir, target_dir)

# 批量下载pdf

In [ ]:
# import os
# from urllib.parse import urlparse

# def generate_pdf_filename(url):
#     """从指定格式的URL生成标准化文件名"""
#     filename = os.path.basename(url).split('.')[0]
#     print(filename)

#     # 解析URL路径
#     path = urlparse(url).path  # 示例：'/finalpage/2024-03-29/1219447098.PDF'
#     print(path)

#     # 分割路径获取日期和文件名
#     path_parts = path.split('/')  # 示例：['', 'finalpage', '2024-03-29', '1219447098.PDF']
#     print(path_parts)

#     # 提取日期部分（路径第3段）
#     date_str = path_parts[-2]  # 示例：'2024-03-29'
#     print(date_str)

#     # 提取公告ID（路径第4段，去除扩展名）
#     filename_part = path_parts[-1]
#     print(filename_part)
#     announcement_id = os.path.splitext(filename_part)[0]  # 示例：'1219447098'

#     # 生成标准化文件名
#     return f"{date_str}_{announcement_id}"

# # 使用示例
# url = "http://static.cninfo.com.cn/finalpage/2024-03-29/1219447098.PDF"
# filename = generate_pdf_filename(url)
# print(filename)  # 输出：2024-03-29-1219447098.pdf


1219447098
/finalpage/2024-03-29/1219447098.PDF
['', 'finalpage', '2024-03-29', '1219447098.PDF']
2024-03-29
1219447098.PDF
2024-03-29_1219447098


In [ ]:
import os
import requests
import threading
from urllib.parse import urlparse
from tqdm import tqdm
from concurrent.futures import ThreadPoolExecutor, as_completed

class ThreadSafeCounter:
    """线程安全计数器"""
    def __init__(self):
        self._value = 0
        self._lock = threading.Lock()

    def increment(self):
        with self._lock:
            self._value += 1

    @property
    def value(self):
        with self._lock:
            return self._value

def download_single_pdf(url, target_folder, error_queue, counter, txt_name):
    """单个PDF下载线程（新增txt_name参数）"""
    try:
        path = urlparse(url).path
        path_parts = path.split('/')
        if len(path_parts) < 4:
            raise ValueError("URL路径格式错误")

        date_str = path_parts[-2]
        base_name = os.path.splitext(path_parts[-1])[0]

        # 添加TXT文件名前缀
        filename = f"{txt_name}_{date_str}-{base_name}.pdf"  # 新增文件名部分
        save_path = os.path.join(target_folder, filename)

        if os.path.exists(save_path):
            return

        with requests.get(url, stream=True, timeout=(6.1, 30)) as r:
            r.raise_for_status()
            with open(save_path, 'wb') as f:
                for chunk in r.iter_content(chunk_size=8192):
                    if chunk:
                        f.write(chunk)

        counter.increment()

    except Exception as e:
        error_queue.put(f"{url} | 错误：{type(e).__name__}-{str(e)}")

def download_pdfs_from_txts(source_folder: str, target_folder: str, max_workers=20):
    """
    多线程下载PDF主函数
    :param source_folder: TXT文件目录
    :param target_folder: PDF保存目录
    :param max_workers: 并发线程数（建议10-30）
    """
    # 初始化线程安全组件
    error_queue = Queue()
    counter = ThreadSafeCounter()

    # 创建目标目录
    os.makedirs(target_folder, exist_ok=True)

    # 收集所有下载链接
    # 修改URL收集逻辑
    all_urls = []
    for txt_file in os.listdir(source_folder):
        if not txt_file.endswith('.txt') or txt_file.startswith('~$'):
            continue

        # 获取TXT文件名（不含扩展名）
        txt_name = os.path.splitext(txt_file)[0]

        with open(os.path.join(source_folder, txt_file), 'r', encoding='utf-8') as f:
            urls = list({line.strip() for line in f if line.strip()})
            # 存储元组（URL，TXT文件名）
            all_urls.extend( [(url, txt_name) for url in urls] )  # 修改这里

    # 修改任务提交部分
    with tqdm(total=len(all_urls), desc="总进度") as pbar:
        with ThreadPoolExecutor(max_workers=max_workers) as executor:
            futures = {
                executor.submit(
                    download_single_pdf,
                    url,           # URL参数
                    target_folder,
                    error_queue,
                    counter,
                    txt_name       # 新增TXT文件名参数
                ): (url, txt_name) for (url, txt_name) in all_urls  # 遍历元组
            }

            for _ in as_completed(futures):
                pbar.update(1)

    # 输出统计信息
    print(f"\n下载完成！成功下载 {counter.value}/{len(all_urls)} 个文件")
    print(f"保存目录：{os.path.abspath(target_folder)}")

    # 输出错误日志
    if not error_queue.empty():
        print("\n错误日志（最多显示20条）：")
        for _ in range(min(20, error_queue.qsize())):
            print(f"• {error_queue.get()}")


In [ ]:
input_folder = "/content/drive/MyDrive/513/HuShen300_pdf_download_links/"
output_folder = "/content/drive/MyDrive/513/HuShen300_pdfs_named/"

download_pdfs_from_txts(input_folder, output_folder)

In [ ]:
input_folder = "/content/drive/MyDrive/513/A50_pdf_download_links/"
output_folder = "/content/drive/MyDrive/513/A50_pdfs_named/"

download_pdfs_from_txts(input_folder, output_folder)

总进度: 100%|██████████| 911/911 [03:42<00:00,  4.09it/s]


下载完成！成功下载 910/911 个文件
保存目录：/content/drive/MyDrive/513/A50_pdfs_named

错误日志（最多显示20条）：
• http://static.cninfo.com.cn/finalpage/2017-04-28/1203410325.PDF | 错误：ReadTimeout-HTTPConnectionPool(host='static.cninfo.com.cn', port=80): Read timed out. (read timeout=30)


In [ ]:
from google.colab import runtime
runtime.unassign()